In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install roboflow


# Install dependencies
!pip install roboflow scikit-learn pandas

import os, cv2, math, numpy as np, pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Activation,
    UpSampling2D, Concatenate, Dropout,
    Input, Lambda, Multiply
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
)
from tensorflow.keras.metrics import MeanIoU
from sklearn.model_selection import KFold
from roboflow import Roboflow

# ─────────────────────────────────────────
# 0) Download Roboflow YOLO dataset
# ─────────────────────────────────────────
from roboflow import Roboflow
rf = Roboflow(api_key="qz3gXKPbOfhvVaI8oDt4")
project = rf.workspace("rkm-nnbdx").project("3d-reconstruction-ga5qp")
version = project.version(1)
dataset = version.download("yolov7") # images + labels/*.txt

BASE       = dataset.location            # e.g. "/kaggle/working/Microfocus-8"
IMG_SIZE   = (384, 384)
BATCH_SIZE = 4
AUTOTUNE   = tf.data.AUTOTUNE
EPOCHS     = 100
SEED       = 42
LR         = 1e-4
FOLDS      = 2
SMOOTH_W   = 7

tf.random.set_seed(SEED)
np.random.seed(SEED)

# ─────────────────────────────────────────
# 1) Generate PNG masks from YOLO polygons
# ─────────────────────────────────────────
def yolo_poly_to_mask(img_path, label_path, out_path, size=IMG_SIZE):
    w,h = size
    mask = np.zeros((h, w), dtype=np.uint8)
    if os.path.exists(label_path):
        for line in open(label_path):
            pts = np.array(list(map(float, line.split()[1:])), dtype=np.float32).reshape(-1,2)
            pts[:,0] *= w; pts[:,1] *= h
            cv2.fillPoly(mask, [pts.astype(np.int32)], 255)
    cv2.imwrite(out_path, mask)

for split in ("train","valid","test"):
    img_dir = os.path.join(BASE, split, "images")
    lbl_dir = os.path.join(BASE, split, "labels")
    msk_dir = os.path.join(BASE, split, "masks")
    os.makedirs(msk_dir, exist_ok=True)
    for img_p in tqdm(glob(f"{img_dir}/*.jpg"), desc=f"Make masks for {split}"):
        fn    = os.path.splitext(os.path.basename(img_p))[0]
        lbl_p = os.path.join(lbl_dir, fn + ".txt")
        out_p = os.path.join(msk_dir, fn + ".png")
        yolo_poly_to_mask(img_p, lbl_p, out_p)

# ─────────────────────────────────────────
# 2) Data pipeline utilities
# ─────────────────────────────────────────
def load_pair(img_path, mask_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, 3)
    img = tf.image.resize(img, IMG_SIZE) / 255.0

    m = tf.io.read_file(mask_path)
    m = tf.image.decode_png(m, 1)
    m = tf.image.resize(m, IMG_SIZE, method="nearest")
    m = tf.cast(m > 127, tf.float32)
    return img, m

def augment(img, m):
    cat = tf.concat([img, m], axis=-1)
    if tf.random.uniform([]) > 0.5: cat = tf.image.flip_left_right(cat)
    if tf.random.uniform([]) > 0.5: cat = tf.image.flip_up_down(cat)
    k = tf.random.uniform([], 0, 4, tf.int32)
    cat = tf.image.rot90(cat, k)
    im, ma = cat[...,:3], cat[...,3:]
    im = tf.image.random_brightness(im, 0.1)
    im = tf.image.random_contrast(im, 0.9, 1.1)
    ma = tf.cast(ma>0.5, tf.float32)
    return im, ma

# **Single** make_ds signature:
def make_ds(img_list, mask_list, train=False):
    ds = tf.data.Dataset.from_tensor_slices((img_list, mask_list))
    ds = ds.map(load_pair, num_parallel_calls=AUTOTUNE)
    if train:
        ds = ds.map(augment, num_parallel_calls=AUTOTUNE)
        ds = ds.shuffle(200, seed=SEED).batch(BATCH_SIZE).repeat()
    else:
        ds = ds.batch(BATCH_SIZE)
    return ds.prefetch(AUTOTUNE)

# Build train/val/test sets (not used in backbone CV, but here for sanity)
train_ds = make_ds(glob(f"{BASE}/train/images/*.jpg"),
                   [p.replace("/images/","/masks/").replace(".jpg",".png")
                    for p in glob(f"{BASE}/train/images/*.jpg")],
                   train=True)
val_ds   = make_ds(glob(f"{BASE}/valid/images/*.jpg"),
                   [p.replace("/images/","/masks/").replace(".jpg",".png")
                    for p in glob(f"{BASE}/valid/images/*.jpg")],
                   train=False)
test_ds  = make_ds(glob(f"{BASE}/test/images/*.jpg"),
                   [p.replace("/images/","/masks/").replace(".jpg",".png")
                    for p in glob(f"{BASE}/test/images/*.jpg")],
                   train=False)

# ─────────────────────────────────────────
# 3) Loss & metrics
# ─────────────────────────────────────────
def dice_loss(y_true,y_pred, smooth=1e-6):
    yt, yp = K.flatten(y_true), K.flatten(y_pred)
    inter = K.sum(yt*yp)
    return 1 - (2*inter+smooth)/(K.sum(yt)+K.sum(yp)+smooth)

def tversky(y_true,y_pred, alpha=0.3,beta=0.7,smooth=1e-6):
    yt, yp = K.flatten(y_true), K.flatten(y_pred)
    tp = K.sum(yt*yp); fp = K.sum((1-yt)*yp); fn = K.sum(yt*(1-yp))
    return (tp+smooth)/(tp+alpha*fp+beta*fn+smooth)

def focal_tversky(y_true,y_pred):
    t = tversky(y_true,y_pred)
    return K.pow((1-t),0.75)

def combined_loss(y_true,y_pred):
    return 0.4*dice_loss(y_true,y_pred) + 0.6*focal_tversky(y_true,y_pred)

metrics = ['accuracy', MeanIoU(num_classes=2,name='mean_iou')]

# ─────────────────────────────────────────
# 4) Attention‑U‑Net builder
# ─────────────────────────────────────────
def attention_gate(skip, g, inter):
    θ = Conv2D(inter,1,padding='same')(skip)
    φ = Conv2D(inter,1,padding='same')(g)
    up= Lambda(lambda x: tf.image.resize(x[0], tf.shape(x[1])[1:3]))([φ, skip])
    f = Activation('relu')(θ+up)
    ψ = Conv2D(1,1,padding='same',activation='sigmoid')(f)
    return Multiply()([skip, ψ])

def build_effattunet(backbone):
    inp = Input((*IMG_SIZE,3))
    enc = backbone(include_top=False, weights='imagenet', input_tensor=inp)
    skips = [enc.get_layer(n).output for n in (
      "block2a_expand_activation",
      "block3a_expand_activation",
      "block4a_expand_activation",
      "block6a_expand_activation"
    )]
    x = enc.output
    filters=[256,128,64,32]
    for i, skip in enumerate(reversed(skips)):
        x = UpSampling2D()(x)
        ag = attention_gate(skip, x, inter=filters[i]//2)
        x  = Concatenate()([x, ag])
        x  = Conv2D(filters[i],3,padding='same',activation='relu')(x)
        x  = BatchNormalization()(x)
        x  = Conv2D(filters[i],3,padding='same',activation='relu')(x)
        x  = BatchNormalization()(x)
        x  = Dropout(0.3)(x)
    # final upsampling to full resolution
    x = UpSampling2D()(x)
    x = Conv2D(32,3,padding='same',activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(32,3,padding='same',activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    out = Conv2D(1,1,activation='sigmoid')(x)
    return Model(inp,out)

# ─────────────────────────────────────────
# 5) Cross‑validation per backbone
# ─────────────────────────────────────────
backbones = {
  # 'EffB1': tf.keras.applications.EfficientNetB1,
  # 'EffB2': tf.keras.applications.EfficientNetB2,
  'EffB5': tf.keras.applications.EfficientNetB5
}

all_imgs  = sorted(glob(f"{BASE}/train/images/*.jpg"))
all_masks = [p.replace("/images/","/masks/").replace(".jpg",".png")
             for p in all_imgs]
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

results = {name: [] for name in backbones}
for name, eff in backbones.items():
    print(f"\n=== {name} ===")
    for fold, (trIdx, valIdx) in enumerate(kf.split(all_imgs)):
        print(f" Fold {fold+1}")
        trI = [all_imgs[i] for i in trIdx];  trM = [all_masks[i] for i in trIdx]
        vI  = [all_imgs[i] for i in valIdx]; vM  = [all_masks[i] for i in valIdx]

        dsTr  = make_ds(trI, trM, train=True)
        dsVal = make_ds(vI, vM, train=False)

        model = build_effattunet(eff)
        model.compile(optimizer=Adam(LR),
                      loss=combined_loss,
                      metrics=metrics)

        def cos_lr(e):
            t = e % 30
            return LR * 0.5 * (1 + math.cos(math.pi * t/30))
        cbs = [
          LearningRateScheduler(cos_lr, verbose=0),
          ModelCheckpoint( f"{name}_f{fold}.h5",
                           monitor="val_mean_iou", mode="max",
                           save_best_only=True),
          ReduceLROnPlateau( monitor="val_mean_iou",
                            factor=0.5, patience=30,
                            mode="max", verbose=0)
        ]

        steps = len(trI)//BATCH_SIZE
        vstps = len(vI)//BATCH_SIZE
        h = model.fit(dsTr,
                      epochs=EPOCHS,
                      steps_per_epoch=steps,
                      validation_data=dsVal,
                      validation_steps=vstps,
                      callbacks=cbs,
                      verbose=1)
        model.load_weights(f"{name}_f{fold}.h5")
        results[name].append(max(h.history['val_mean_iou']))

    print(" →", name, "mean val mIoU:", np.mean(results[name]))

# ─────────────────────────────────────────
# 6) Plot backbone comparison
# ─────────────────────────────────────────
plt.figure(figsize=(6,4))
names  = list(results.keys())
scores = [np.mean(results[n]) for n in names]
plt.bar(names, scores, color=['#a6cee3','#1f78b4','#b2df8a'])
plt.ylabel("Mean Val mIoU")
plt.ylim(0,1)
for i, v in enumerate(scores):
    plt.text(i, v+0.01, f"{v:.3f}", ha='center')
plt.title("Backbone comparison: EffB1 vs EffB2 vs EffB3")
plt.show()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 36.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
google-cola

2025-06-25 04:09:33.996421: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750824574.232468      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750824574.293804      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


loading Roboflow workspace...
loading Roboflow project...


Make masks for test: 100%|██████████| 179/179 [00:00<00:00, 1486.84it/s]
I0000 00:00:1750824591.739489      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1750824591.740207      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5



=== EffB3 ===
 Fold 1
43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100


I0000 00:00:1750824698.303578     110 service.cc:148] XLA service 0x7ddd440027e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750824698.304731     110 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1750824698.304752     110 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1750824707.774968     110 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1750824715.831487     110 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750824715.976937     110 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750824727.951438     110 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. Th

403/403 ━━━━━━━━━━━━━━━━━━━━ 314s 337ms/step - accuracy: 0.7870 - loss: 0.3542 - mean_iou: 0.3533 - val_accuracy: 0.7239 - val_loss: 0.7076 - val_mean_iou: 0.3332 - learning_rate: 1.0000e-04
Epoch 2/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 118s 294ms/step - accuracy: 0.8560 - loss: 0.2634 - mean_iou: 0.3574 - val_accuracy: 0.8543 - val_loss: 0.4127 - val_mean_iou: 0.3332 - learning_rate: 9.9726e-05
Epoch 3/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 118s 293ms/step - accuracy: 0.8607 - loss: 0.2601 - mean_iou: 0.3653 - val_accuracy: 0.4573 - val_loss: 0.5226 - val_mean_iou: 0.3332 - learning_rate: 9.8907e-05
Epoch 4/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 118s 292ms/step - accuracy: 0.8732 - loss: 0.2366 - mean_iou: 0.3868 - val_accuracy: 0.8776 - val_loss: 0.3772 - val_mean_iou: 0.3332 - learning_rate: 9.7553e-05
Epoch 5/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 118s 293ms/step - accuracy: 0.8833 - loss: 0.2224 - mean_iou: 0.3984 - val_accuracy: 0.3628 - val_loss: 0.5731 - val_mean_iou: 0.3332 - learning_rate: 9.5677e-

In [ ]:
# ─────────────────────────────────────────
# 6) Plot backbone comparison
# ─────────────────────────────────────────
plt.figure(figsize=(6,4))
names  = list(results.keys())
scores = [np.mean(results[n]) for n in names]
plt.bar(names, scores, color=['#a6cee3','#1f78b4','#b2df8a'])
plt.ylabel("Mean Val mIoU")
plt.ylim(0,1)
for i, v in enumerate(scores):
    plt.text(i, v+0.01, f"{v:.3f}", ha='center')
plt.title("Backbone comparison: EffB1 vs EffB2 vs EffB3")
plt.show()


In [ ]:
# Install dependencies
!pip install roboflow scikit-learn pandas

import os, cv2, math, numpy as np, pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Activation,
    UpSampling2D, Concatenate, Dropout,
    Input, Lambda, Multiply
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
)
from tensorflow.keras.metrics import MeanIoU
from sklearn.model_selection import KFold
from roboflow import Roboflow

# ─────────────────────────────────────────
# 0) Download Roboflow YOLO dataset
# ─────────────────────────────────────────
rf = Roboflow(api_key="qz3gXKPbOfhvVaI8oDt4")
project = rf.workspace("rkm-nnbdx").project("microfocus")
version = project.version(8)
dataset = version.download("yolov7")  # images + labels/*.txt

BASE       = dataset.location            # e.g. "/kaggle/working/Microfocus-8"
IMG_SIZE   = (384, 384)
BATCH_SIZE = 8
AUTOTUNE   = tf.data.AUTOTUNE
EPOCHS     = 100
SEED       = 42
LR         = 1e-4
FOLDS      = 2
SMOOTH_W   = 7

tf.random.set_seed(SEED)
np.random.seed(SEED)

# ─────────────────────────────────────────
# 1) Generate PNG masks from YOLO polygons
# ─────────────────────────────────────────
def yolo_poly_to_mask(img_path, label_path, out_path, size=IMG_SIZE):
    w,h = size
    mask = np.zeros((h, w), dtype=np.uint8)
    if os.path.exists(label_path):
        for line in open(label_path):
            pts = np.array(list(map(float, line.split()[1:])), dtype=np.float32).reshape(-1,2)
            pts[:,0] *= w; pts[:,1] *= h
            cv2.fillPoly(mask, [pts.astype(np.int32)], 255)
    cv2.imwrite(out_path, mask)

for split in ("train","valid","test"):
    img_dir = os.path.join(BASE, split, "images")
    lbl_dir = os.path.join(BASE, split, "labels")
    msk_dir = os.path.join(BASE, split, "masks")
    os.makedirs(msk_dir, exist_ok=True)
    for img_p in tqdm(glob(f"{img_dir}/*.jpg"), desc=f"Make masks for {split}"):
        fn    = os.path.splitext(os.path.basename(img_p))[0]
        lbl_p = os.path.join(lbl_dir, fn + ".txt")
        out_p = os.path.join(msk_dir, fn + ".png")
        yolo_poly_to_mask(img_p, lbl_p, out_p)

# ─────────────────────────────────────────
# 2) Data pipeline utilities
# ─────────────────────────────────────────
def load_pair(img_path, mask_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, 3)
    img = tf.image.resize(img, IMG_SIZE) / 255.0

    m = tf.io.read_file(mask_path)
    m = tf.image.decode_png(m, 1)
    m = tf.image.resize(m, IMG_SIZE, method="nearest")
    m = tf.cast(m > 127, tf.float32)
    return img, m

def augment(img, m):
    cat = tf.concat([img, m], axis=-1)
    if tf.random.uniform([]) > 0.5: cat = tf.image.flip_left_right(cat)
    if tf.random.uniform([]) > 0.5: cat = tf.image.flip_up_down(cat)
    k = tf.random.uniform([], 0, 4, tf.int32)
    cat = tf.image.rot90(cat, k)
    im, ma = cat[...,:3], cat[...,3:]
    im = tf.image.random_brightness(im, 0.1)
    im = tf.image.random_contrast(im, 0.9, 1.1)
    ma = tf.cast(ma>0.5, tf.float32)
    return im, ma

# **Single** make_ds signature:
def make_ds(img_list, mask_list, train=False):
    ds = tf.data.Dataset.from_tensor_slices((img_list, mask_list))
    ds = ds.map(load_pair, num_parallel_calls=AUTOTUNE)
    if train:
        ds = ds.map(augment, num_parallel_calls=AUTOTUNE)
        ds = ds.shuffle(200, seed=SEED).batch(BATCH_SIZE).repeat()
    else:
        ds = ds.batch(BATCH_SIZE)
    return ds.prefetch(AUTOTUNE)

# Build train/val/test sets (not used in backbone CV, but here for sanity)
train_ds = make_ds(glob(f"{BASE}/train/images/*.jpg"),
                   [p.replace("/images/","/masks/").replace(".jpg",".png")
                    for p in glob(f"{BASE}/train/images/*.jpg")],
                   train=True)
val_ds   = make_ds(glob(f"{BASE}/valid/images/*.jpg"),
                   [p.replace("/images/","/masks/").replace(".jpg",".png")
                    for p in glob(f"{BASE}/valid/images/*.jpg")],
                   train=False)
test_ds  = make_ds(glob(f"{BASE}/test/images/*.jpg"),
                   [p.replace("/images/","/masks/").replace(".jpg",".png")
                    for p in glob(f"{BASE}/test/images/*.jpg")],
                   train=False)

# ─────────────────────────────────────────
# 3) Loss & metrics
# ─────────────────────────────────────────
def dice_loss(y_true,y_pred, smooth=1e-6):
    yt, yp = K.flatten(y_true), K.flatten(y_pred)
    inter = K.sum(yt*yp)
    return 1 - (2*inter+smooth)/(K.sum(yt)+K.sum(yp)+smooth)

def tversky(y_true,y_pred, alpha=0.3,beta=0.7,smooth=1e-6):
    yt, yp = K.flatten(y_true), K.flatten(y_pred)
    tp = K.sum(yt*yp); fp = K.sum((1-yt)*yp); fn = K.sum(yt*(1-yp))
    return (tp+smooth)/(tp+alpha*fp+beta*fn+smooth)

def focal_tversky(y_true,y_pred):
    t = tversky(y_true,y_pred)
    return K.pow((1-t),0.75)

def combined_loss(y_true,y_pred):
    return 0.4*dice_loss(y_true,y_pred) + 0.6*focal_tversky(y_true,y_pred)

metrics = ['accuracy', MeanIoU(num_classes=2,name='mean_iou')]

# ─────────────────────────────────────────
# 4) Attention‑U‑Net builder
# ─────────────────────────────────────────
def attention_gate(skip, g, inter):
    θ = Conv2D(inter,1,padding='same')(skip)
    φ = Conv2D(inter,1,padding='same')(g)
    up= Lambda(lambda x: tf.image.resize(x[0], tf.shape(x[1])[1:3]))([φ, skip])
    f = Activation('relu')(θ+up)
    ψ = Conv2D(1,1,padding='same',activation='sigmoid')(f)
    return Multiply()([skip, ψ])

def build_effattunet(backbone):
    inp = Input((*IMG_SIZE,3))
    enc = backbone(include_top=False, weights='imagenet', input_tensor=inp)
    skips = [enc.get_layer(n).output for n in (
      "block2a_expand_activation",
      "block3a_expand_activation",
      "block4a_expand_activation",
      "block6a_expand_activation"
    )]
    x = enc.output
    filters=[256,128,64,32]
    for i, skip in enumerate(reversed(skips)):
        x = UpSampling2D()(x)
        ag = attention_gate(skip, x, inter=filters[i]//2)
        x  = Concatenate()([x, ag])
        x  = Conv2D(filters[i],3,padding='same',activation='relu')(x)
        x  = BatchNormalization()(x)
        x  = Conv2D(filters[i],3,padding='same',activation='relu')(x)
        x  = BatchNormalization()(x)
        x  = Dropout(0.3)(x)
    # final upsampling to full resolution
    x = UpSampling2D()(x)
    x = Conv2D(32,3,padding='same',activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(32,3,padding='same',activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    out = Conv2D(1,1,activation='sigmoid')(x)
    return Model(inp,out)

# ─────────────────────────────────────────
# 5) Cross‑validation per backbone
# ─────────────────────────────────────────
backbones = {
  'EffB1': tf.keras.applications.EfficientNetB1,
  'EffB2': tf.keras.applications.EfficientNetB2,
  'EffB3': tf.keras.applications.EfficientNetB3
}

all_imgs  = sorted(glob(f"{BASE}/train/images/*.jpg"))
all_masks = [p.replace("/images/","/masks/").replace(".jpg",".png")
             for p in all_imgs]
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

results = {name: [] for name in backbones}
for name, eff in backbones.items():
    print(f"\n=== {name} ===")
    for fold, (trIdx, valIdx) in enumerate(kf.split(all_imgs)):
        print(f" Fold {fold+1}")
        trI = [all_imgs[i] for i in trIdx];  trM = [all_masks[i] for i in trIdx]
        vI  = [all_imgs[i] for i in valIdx]; vM  = [all_masks[i] for i in valIdx]

        dsTr  = make_ds(trI, trM, train=True)
        dsVal = make_ds(vI, vM, train=False)

        model = build_effattunet(eff)
        model.compile(optimizer=Adam(LR),
                      loss=combined_loss,
                      metrics=metrics)

        def cos_lr(e):
            t = e % 30
            return LR * 0.5 * (1 + math.cos(math.pi * t/30))
        cbs = [
          LearningRateScheduler(cos_lr, verbose=0),
          ModelCheckpoint( f"{name}_f{fold}.h5",
                           monitor="val_mean_iou", mode="max",
                           save_best_only=True),
          ReduceLROnPlateau( monitor="val_mean_iou",
                            factor=0.5, patience=10,
                            mode="max", verbose=0)
        ]

        steps = len(trI)//BATCH_SIZE
        vstps = len(vI)//BATCH_SIZE
        h = model.fit(dsTr,
                      epochs=EPOCHS,
                      steps_per_epoch=steps,
                      validation_data=dsVal,
                      validation_steps=vstps,
                      callbacks=cbs,
                      verbose=1)
        model.load_weights(f"{name}_f{fold}.h5")
        results[name].append(max(h.history['mean_iou']))

    print(" →", name, "mean IoU:", np.mean(results[name]))

# ─────────────────────────────────────────
# 6) Plot backbone comparison
# ─────────────────────────────────────────
plt.figure(figsize=(6,4))
names  = list(results.keys())
scores = [np.mean(results[n]) for n in names]
plt.bar(names, scores, color=['#a6cee3','#1f78b4','#b2df8a'])
plt.ylabel("mIoU")
plt.ylim(0,1)
for i, v in enumerate(scores):
    plt.text(i, v+0.01, f"{v:.3f}", ha='center')
plt.title("Backbone comparison: EffB1 vs EffB2 vs EffB3")
plt.show()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 35.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
google-cola

2025-06-22 07:11:13.999996: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750576274.202637      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750576274.260018      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


loading Roboflow workspace...
loading Roboflow project...


Make masks for test: 100%|██████████| 72/72 [00:00<00:00, 1278.81it/s]
I0000 00:00:1750576290.135178      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



=== EffB1 ===
 Fold 1
27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/100


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# 1) Plot Training vs Validation Curves
# --------------------------------------
epochs = range(1, len(h.history['loss']) + 1)

plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.plot(epochs, h.history['loss'],    label='Train Loss')
plt.plot(epochs, h.history['val_loss'],label='Val Loss')
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1,3,2)
plt.plot(epochs, h.history['accuracy'],     label='Train Acc')
plt.plot(epochs, h.history['val_accuracy'], label='Val Acc')
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Acc")
plt.legend()

plt.subplot(1,3,3)
plt.plot(epochs, h.history['mean_iou'],      label='Train mIoU')
plt.plot(epochs, h.history['val_mean_iou'],  label='Val mIoU')
plt.title("Mean IoU")
plt.xlabel("Epoch")
plt.ylabel("mIoU")
plt.legend()

plt.tight_layout()
plt.show()


# # 2) Compute & Plot ROC on the Test Set
# # -------------------------------------
# y_trues = []
# y_probs = []

# for imgs, masks in test_ds:
#     preds = best.predict(imgs, verbose=0)  # shape (B, H, W, 1)
#     y_trues.append(masks.numpy().ravel())
#     y_probs.append(preds.ravel())

# y_true = np.concatenate(y_trues)
# y_score = np.concatenate(y_probs)

# # If your test set is large you might subsample:
# # idx = np.random.choice(len(y_true), size=200_000, replace=False)
# # y_true, y_score = y_true[idx], y_score[idx]

# fpr, tpr, _ = roc_curve(y_true, y_score)
# roc_auc = auc(fpr, tpr)

# plt.figure(figsize=(6,6))
# plt.plot(fpr, tpr, lw=2, label=f"ROC AUC = {roc_auc:.3f}")
# plt.plot([0,1],[0,1], linestyle='--', color='gray')
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.title("ROC Curve (pixel‑wise)")
# plt.legend(loc="lower right")
# plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import MeanIoU

# 1. Define your custom losses and metrics used during training
def dice_loss(y_true,y_pred, smooth=1e-6):
    yt, yp = tf.keras.backend.flatten(y_true), tf.keras.backend.flatten(y_pred)
    inter = tf.keras.backend.sum(yt*yp)
    return 1 - (2*inter+smooth)/(tf.keras.backend.sum(yt)+tf.keras.backend.sum(yp)+smooth)

def tversky(y_true,y_pred, alpha=0.3,beta=0.7,smooth=1e-6):
    yt, yp = tf.keras.backend.flatten(y_true), tf.keras.backend.flatten(y_pred)
    tp = tf.keras.backend.sum(yt*yp)
    fp = tf.keras.backend.sum((1-yt)*yp)
    fn = tf.keras.backend.sum(yt*(1-yp))
    return (tp+smooth)/(tp+alpha*fp+beta*fn+smooth)

def focal_tversky(y_true,y_pred):
    t = tversky(y_true,y_pred)
    return tf.keras.backend.pow((1-t),0.75)

def combined_loss(y_true,y_pred):
    return 0.4*dice_loss(y_true,y_pred) + 0.6*focal_tversky(y_true,y_pred)

# 2. Load best trained model (EffB3, fold 1 for example)
best_model_path = "EffB3_f1.h5"
model = load_model(best_model_path,
                   custom_objects={
                       "combined_loss": combined_loss,
                       "MeanIoU": MeanIoU,
                       "dice_loss": dice_loss,
                       "tversky": tversky,
                       "focal_tversky": focal_tversky
                   })

# 3. Load training history if you have it saved, or use the variable `h` if in memory
# Example: assume h.history is available
# You can also save history during training using: `pd.DataFrame(h.history).to_csv(...)`

# If you already have `h = model.fit(...)` earlier:
history = h.history  # replace this if loading from CSV: pd.read_csv('history.csv')

# 4. Smooth helper
def smooth(values, weight=0.8):
    smoothed = []
    last = values[0]
    for v in values:
        smoothed_val = last * weight + (1 - weight) * v
        smoothed.append(smoothed_val)
        last = smoothed_val
    return smoothed

# 5. Plot curves
epochs = range(1, len(history['loss']) + 1)

plt.figure(figsize=(18,5))

# Loss
plt.subplot(1, 3, 1)
plt.plot(epochs, smooth(history['loss']), label='Train Loss (smoothed)')
plt.plot(epochs, smooth(history['val_loss']), label='Val Loss (smoothed)')
plt.title("Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

# Accuracy
plt.subplot(1, 3, 2)
plt.plot(epochs, smooth(history['accuracy']), label='Train Acc (smoothed)')
plt.plot(epochs, smooth(history['val_accuracy']), label='Val Acc (smoothed)')
plt.title("Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# mIoU
plt.subplot(1, 3, 3)
plt.plot(epochs, smooth(history['mean_iou']), label='Train mIoU (smoothed)')
plt.plot(epochs, smooth(history['val_mean_iou']), label='Val mIoU (smoothed)')
plt.title("Mean IoU per Epoch")
plt.xlabel("Epoch")
plt.ylabel("mIoU")
plt.legend()

plt.tight_layout()
plt.show()
